In [ ]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
import time

class VideoYOLODetection:
    def __init__(self, video_path, model_path='yolov5s.pt'):
        self.model = YOLO(model_path)
        self.cap = cv2.VideoCapture(video_path)
        self.colors = np.random.randint(0, 255, size=(80, 3), dtype=np.uint8)

    def run(self, output_path='output.mp4'):
        width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = self.cap.get(cv2.CAP_PROP_FPS)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            results = self.model(frame)

            for result in results:
                boxes = result.boxes.cpu().numpy()
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].astype(int)
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    label = f"{result.names[cls]} {conf:.2f}"
                    color = [int(c) for c in self.colors[cls]]
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            out.write(frame)

        self.cap.release()
        out.release()

# Usage
detector = VideoYOLODetection('/content/video_capture.mp4') 
detector.run()